In [1]:
!pip install google-colab-shell
from google_colab_shell import getshell
getshell()


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4132 sha256=7144585a111fec7286e0b5abae12fe523c965b974cd71fa56c0f0b665e24bb74
  Stored in directory: c:\users\chxss\appdata\local\pip\cache\wheels\14\b3\39\fd3604d9e5f2b9c00239248aa62f4cfe8ef51972220862b034
Successfully built google-colab-shell


ModuleNotFoundError: No module named 'google'

#### 목표
- 공부시간에 따른 학생의 성적 예측하는 모델 만들기
- sklearn을 이용한 모델링과 keras를 이용한 모델링

#### 데이터로딩

In [2]:
%cd "C:\Users\chxss\OneDrive\Desktop\AiSchool\python\ML\DeepLearning"

C:\Users\chxss\OneDrive\Desktop\AiSchool\python\ML\DeepLearning


C:\Users\chxss\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv("./data/student-mat.csv", delimiter=";")
data

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


| 컬럼 이름       | 설명                             |
|-----------------|----------------------------------|
| school          | 학교 (GP - Gabriel Pereira, MS - Mousinho da Silveira) |
| sex             | 성별 (F - Female, M - Male)      |
| age             | 나이                             |
| address         | 주소 유형 (U - Urban, R - Rural) |
| famsize         | 가족 크기 (LE3 - 3명 이하, GT3 - 3명 초과) |
| Pstatus         | 부모 동거 상태 (T - 함께, A - 별거) |
| Medu            | 어머니 교육 수준 (0 - 없음, 1 - 초등, 2 - 중등, 3 - 고등, 4 - 대졸) |
| Fedu            | 아버지 교육 수준 (0 - 없음, 1 - 초등, 2 - 중등, 3 - 고등, 4 - 대졸) |
| Mjob            | 어머니 직업 (teacher, health, services, at_home, other) |
| Fjob            | 아버지 직업 (teacher, health, services, at_home, other) |
| reason          | 학교 선택 이유 (home, reputation, course, other) |
| guardian        | 보호자 (mother, father, other)  |
| traveltime      | 통학 시간 (1 - 15분 이하, 2 - 15~30분, 3 - 30분~1시간, 4 - 1시간 초과) |
| studytime       | 주간 공부 시간 (1 - 2시간 이하, 2 - 2~5시간, 3 - 5~10시간, 4 - 10시간 초과) |
| failures        | 과목 낙제 수 (n if 1<=n<3, else 4) |
| schoolsup       | 학교 추가 지원 (yes, no)        |
| famsup          | 가족 추가 지원 (yes, no)        |
| paid            | 과외 수업 참여 (yes, no)        |
| activities      | 학교 외 활동 참여 (yes, no)     |
| nursery         | 유치원 참여 여부 (yes, no)      |
| higher          | 고등교육 희망 여부 (yes, no)    |
| internet        | 가정 내 인터넷 접속 여부 (yes, no) |
| romantic        | 로맨틱 관계 여부 (yes, no)      |
| famrel          | 가족 관계의 질 (1 - 매우 나쁨, 5 - 매우 좋음) |
| freetime        | 여가 시간의 양 (1 - 매우 적음, 5 - 매우 많음) |
| goout           | 외출 빈도 (1 - 매우 적음, 5 - 매우 많음) |
| Dalc            | 평일 음주량 (1 - 매우 적음, 5 - 매우 많음) |
| Walc            | 주말 음주량 (1 - 매우 적음, 5 - 매우 많음) |
| health          | 현재 건강 상태 (1 - 매우 나쁨, 5 - 매우 좋음) |
| absences        | 결석 일수                        |
| G1              | 1학기 성적                       |
| G2              | 2학기 성적                       |
| G3              | 최종 성적                        |

#### 문제와 답 추출
- 공부시간 -> X
- 최종성적 -> Y

In [5]:
X = data['studytime']
y = data['G3']

In [6]:
X.shape, y.shape

((395,), (395,))

#### 선형회귀 모델링
- 훈련/평가 데이터 분리
- 모델학습 및 평가

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [8]:
# 훈련/평가 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, # 테스트20% 훈련 80%
                                                    random_state=829)

In [9]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(316,) (316,)
(79,) (79,)


In [10]:
# 모델 생성 -> 모델 학습 -> 모델 예측 -> 모델 평가
lr_model = LinearRegression()
lr_model.fit(X_train.values.reshape(-1, 1), y_train)
pre = lr_model.predict(X_test.values.reshape(-1, 1))
score = mean_squared_error(y_test, pre)
print('평균제곱오차 : ',score)

평균제곱오차 :  20.61156468545777


#### Keras를 이용한 딥러닝 모델링

In [12]:
# !pip install tensorflow

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB ? eta 0:00:00
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   --------------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer

In [14]:
# 모델 생성(모델 설계)
# 뼈대 생성
deep_model = Sequential()
# 입력층
deep_model.add(InputLayer(shape=(1,)))
# 중간층(은닉층)
deep_model.add(Dense(units=8, activation='sigmoid'))
deep_model.add(Dense(units=8, activation='sigmoid'))
# 출력층
deep_model.add(Dense(units=1))

In [15]:
# 모델 학습
# 손실 함수, 최적화 함수 => 커스터마이징
deep_model.compile(loss='mse',
                   optimizer='adam')
deep_model.fit(X_train,y_train,
               validation_split=0.2, # 훈련데이터에서 (316개) 20%를 검증용으로 활용
               epochs=20) # 모델의 가중치를 업데이트하는 반복 횟수


Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 127.6131 - val_loss: 129.8818
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 128.3804 - val_loss: 128.3157
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 124.2195 - val_loss: 126.7590
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 123.1618 - val_loss: 125.2106
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 125.9416 - val_loss: 123.6566
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 121.5659 - val_loss: 122.1198
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 111.6177 - val_loss: 120.6044
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 117.9885 - val_loss: 119.0636
Epoch 9/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 107.6042 - val_loss: 117.5500
Epoch 10/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 115.4402 - val_loss: 116.0254
Epoch 11/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 116.0292 - val_loss: 114.5027
Epoch 12/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - 

In [16]:
# 모델 예측 및 평가
deep_model.evaluate(X_test, y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 99.5902   


97.72767639160156